In [1]:
import pandas as pd
import numpy as np
from faker import Faker

fake = Faker('en_IN')

n=10000

hospital_database = pd.DataFrame({
    "Patient id" : range(1, n+1),
    "Patient age": np.random.randint(18, 80, n),
    "Patient city": np.random.choice(['Delhi','Mumbai','Chandigarh','Ambala', 'Rishikesh','Manali','Gurugram'],n),
    "Diseases": np.random.choice(['Diabetes','Blood Pressure', 'Insomnia','Cancer','Obesity','Demensia','Asthma', 
'HIV/AIDS', 'Cholera', 'Polio','COVID-19','Hepatitis (A, B, C)', 'Leprosy','Pneumonia','Goitre',
'Rickets', 'Kwashiorkor', 'Measles', 'Ringworm', 'Chickenpox'], n),
'Appointment Date': np.random.choice(pd.date_range('2025-01-01','2026-01-01'), n),
'Assigned Doctor': np.random.choice(["Dr Ramesh Sinha","Dr Sakshi Malik","Dr Harish Talwar","Dr Kaliya Sonkar","Dr Ali Aanwar" ], n),
'Gender':np.random.choice(['Male','Female','Transgender'],n),
'BMI':np.random.uniform(18.5, 30.0, n).round(1),
'Blood Pressure': np.random.randint(90, 140, n),
'Cholesterol Level': np.random.randint(150, 250, n),
'Blood Sugar Level': np.random.randint(70, 150, n),
'BMI Category': np.random.choice(['Underweight', 'Normal weight', 'Overweight', 'Obese'], n),
"first_name": [fake.first_name() for _ in range(n)],
"last_name": [fake.last_name() for _ in range(n)],
"Address": [fake.address().replace("\n",',')for _ in range(n)],
"Phone Number":[fake.phone_number() for _ in range(n)],
"Blood Grouph":np.random.choice(['B+','A+', 'O', 'AB'], n),
"Height": np.random.randint(140, 200, n),
"Payment Status": np.random.choice(['Paid','Pending'],n)

})

hospital_database.to_csv('Hospital.csv', index=False)

In [10]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Shubham@1999",
    database="hospital_analytics"
)
cursor = conn.cursor()
print("Connected to MySQL database")

Connected to MySQL database


In [3]:
query="SELECT * FROM hospital_raw"

df = pd.read_sql(query, conn)
print(df.head())

C:\Users\shubh\AppData\Local\Temp\ipykernel_23616\984702843.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   patient_id  patient_age patient_city             diseases appointment_date  \
0           1           52       Mumbai              Obesity       2025-11-15   
1           2           49       Manali             HIV/AIDS       2025-11-03   
2           3           24       Ambala              Obesity       2025-07-19   
3           4           18   Chandigarh             Insomnia       2025-10-06   
4           5           47       Manali  Hepatitis (A, B, C)       2025-11-28   

    assigned_doctor       gender   bmi  blood_pressure  cholesterol_level  \
0  Dr Kaliya Sonkar       Female  24.1             109                169   
1   Dr Sakshi Malik         Male  22.5             113                195   
2     Dr Ali Aanwar         Male  26.3             103                178   
3  Dr Harish Talwar       Female  26.0              94                189   
4   Dr Sakshi Malik  Transgender  21.9              99                214   

   ...  first_name        last_name               

In [6]:
df["bp_category"]= pd.cut(df['blood_pressure'], 
bins=[0, 120, 129, 139, np.inf], 
labels=['Normal', 'Elevated', 'Hypertension Stage 1', 'Hypertension Stage 2'])


In [11]:
updated_query = """
update hospital_raw
set bp_category= %s
wHERE patient_id= %s
"""

data = list(zip(df['bp_category'], df['patient_id']))
cursor.executemany(updated_query, data)
conn.commit()


In [12]:
df.columns

Index(['patient_id', 'patient_age', 'patient_city', 'diseases',
       'appointment_date', 'assigned_doctor', 'gender', 'bmi',
       'blood_pressure', 'cholesterol_level', 'blood_sugar_level',
       'bmi_category', 'first_name', 'last_name', 'address', 'phone_number',
       'blood_group', 'height', 'payment_status', 'age_group', 'health_risk',
       'disease_category', 'bp_category'],
      dtype='object')

In [13]:
query="SELECT bp_category, COUNT(*) as count FROM hospital_raw GROUP BY bp_category ORDER BY count DESC"
bp_counts = pd.read_sql(query, conn)
print(bp_counts)

            bp_category  count
0                Normal   6234
1  Hypertension Stage 1   1974
2              Elevated   1792


C:\Users\shubh\AppData\Local\Temp\ipykernel_23616\407678229.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  bp_counts = pd.read_sql(query, conn)
